## 🧩 FASE 1: Diagnóstico de Problemas

In [1]:
!pip install chardet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
### 📌 FASE 1 - TAREA 1: Diagnóstico de Problemas de Codificación

import pandas as pd
import chardet
import re

### Paso 1: Detectar codificación del archivo

# 📂 Ruta del archivo CSV
file_path = 'resenas_multilingue.csv'

# 🔍 Detectar codificación probable
with open(file_path, 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    encoding_detected = result['encoding']
    print(f"Codificación detectada: {encoding_detected}")

print("📦 Codificación detectada por chardet:", result)



Codificación detectada: Windows-1254
📦 Codificación detectada por chardet: {'encoding': 'Windows-1254', 'confidence': 0.4858157129492815, 'language': 'Turkish'}


## 🛠️ FASE 2: Normalización Unicode y Codificación

In [3]:
# Paso 2: Cargar dataset respetando comas dentro de texto (usando quotechar)
import pandas as pd

file_path = 'resenas_multilingue.csv'
encoding_usado = 'ISO-8859-1'  # O la que veas que mejor evita errores

datos = []
with open(file_path, 'r', encoding=encoding_usado) as f:
    encabezado = f.readline().strip().split(',')  # Leer encabezado
    for linea in f:
        linea = linea.strip()
        # Separar desde la derecha en 3 partes
        try:
            texto, categoria = linea.rsplit(',', 1)
            texto, rating = texto.rsplit(',', 1)
            texto, idioma = texto.rsplit(',', 1)
            datos.append({
                'texto': texto,
                'idioma': idioma,
                'rating': int(rating),
                'categoria': categoria
            })
        except ValueError:
            print(f"❌ Línea con formato incorrecto: {linea}")
            continue

# Crear el DataFrame manualmente
df = pd.DataFrame(datos)
print("✅ DataFrame reconstruido correctamente:")
print(df.head(3))



✅ DataFrame reconstruido correctamente:
                                               texto idioma  rating  \
0  Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...     es       2   
1  J'adore cet ordinateur! Mais le clavier est tr...     fr       4   
2  This product is a disaster... don't buy it! #W...     en       1   

       categoria  
0  electrÃ³nicos  
1  electrÃ³nicos  
2  electrÃ³nicos  


In [4]:
df.head(5)

,texto,idioma,rating,categoria
0,Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...,es,2,electrÃ³nicos
1,J'adore cet ordinateur! Mais le clavier est tr...,fr,4,electrÃ³nicos
2,This product is a disaster... don't buy it! #W...,en,1,electrÃ³nicos
3,"La camisa es bonita, pero se encogiÃÂ³ despuÃ...",es,2,ropa
4,Le service clientÃÂ¨le est nul!!! Je ne recom...,fr,1,servicios


In [5]:
import pandas as pd
import unicodedata

# Diccionario de reemplazos (respaldo por si la recodificación no resuelve todo)
errores_comunes = {
    "Ã¡": "á", "Ã©": "é", "Ã­": "í", "Ã³": "ó", "Ãº": "ú",
    "Ã": "Á", "Ã‰": "É", "Ã": "Í", "Ã“": "Ó", "Ãš": "Ú",
    "Ã±": "ñ", "Ã‘": "Ñ", "Ã¼": "ü", "Ãœ": "Ü",
    "Ã¨": "è", "Ãª": "ê", "Ã¢": "â", "Ã®": "î", "Ã´": "ô", "Ã§": "ç",
    "Ã€": "À", "ÃŸ": "ß", "Ã¶": "ö",
    "â": "'", "â": "-", "â": "—", "â¦": "…", 
    "â€œ": "“", "â€": "”", "â€˜": "‘", "â€™": "’"
}

# Emojis relevantes para análisis de sentimiento
emojis_utiles = {"😊", "😞", "😠", "💯", "🔋", "🎧"}

def recodificar(texto):
    try:
        return texto.encode("latin1").decode("utf-8")
    except:
        return texto

# Preservar Emojis Relevantes, eliminar símbolos no esenciales
def eliminar_simbolos_no_utiles(texto):
    return ''.join(ch for ch in texto if not unicodedata.category(ch).startswith("S") or ch in emojis_utiles)

def corregir_texto(texto):
    texto = recodificar(texto)
    for error, correccion in errores_comunes.items():
        texto = texto.replace(error, correccion)
    texto = unicodedata.normalize("NFC", texto) # Normalizar Unicode (e´ → é)
    texto = eliminar_simbolos_no_utiles(texto)
    return texto

# Detectar si hay errores después de corregir
def detectar_textos_corruptos(texto):
    return any(error in texto for error in errores_comunes)

# Aplicar al DataFrame
if 'texto' in df.columns:
    df['texto_corregido'] = df['texto'].astype(str).apply(corregir_texto)
    df['texto_corrupto'] = df['texto_corregido'].astype(str).apply(detectar_textos_corruptos)

    print("\n📊 Resumen del diagnóstico:")
    print(f"Cantidad de textos corruptos detectados: {df['texto_corrupto'].sum()} de {len(df)} filas")

    print("\n🔍 Ejemplos de textos antes y después:")
    print(df[['texto', 'texto_corregido']].head(10))
else:
    print("❗La columna 'texto' no existe en el DataFrame. Revisa el archivo CSV.")



📊 Resumen del diagnóstico:
Cantidad de textos corruptos detectados: 0 de 29 filas

🔍 Ejemplos de textos antes y después:
                                               texto  \
0  Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...   
1  J'adore cet ordinateur! Mais le clavier est tr...   
2  This product is a disaster... don't buy it! #W...   
3  La camisa es bonita, pero se encogiÃÂ³ despuÃ...   
4  Le service clientÃÂ¨le est nul!!! Je ne recom...   
5  I'm impressed with the battery life!!! Works f...   
6  Ãâ¡a marche nickel! Par contre, l'ÃÂ©cran e...   
7  El envÃ­o fue rÃÂ¡pido, pero el producto esta...   
8  Good price, but the material feels cheap. #Not...   
9  La livraison a pris 10 jours... c'est inaccept...   

                                     texto_corregido  
0  Éste celular es increíble!!! Pero tarda 5hs en...  
1  J'adore cet ordinateur! Mais le clavier est tr...  
2  This product is a disaster... don't buy it! #W...  
3  La camisa es bonita, pero se encogió d

In [6]:
df.head(5)

,texto,idioma,rating,categoria,texto_corregido,texto_corrupto
0,Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...,es,2,electrÃ³nicos,Éste celular es increíble!!! Pero tarda 5hs en...,False
1,J'adore cet ordinateur! Mais le clavier est tr...,fr,4,electrÃ³nicos,J'adore cet ordinateur! Mais le clavier est tr...,False
2,This product is a disaster... don't buy it! #W...,en,1,electrÃ³nicos,This product is a disaster... don't buy it! #W...,False
3,"La camisa es bonita, pero se encogiÃÂ³ despuÃ...",es,2,ropa,"La camisa es bonita, pero se encogió después d...",False
4,Le service clientÃÂ¨le est nul!!! Je ne recom...,fr,1,servicios,Le service clientèle est nul!!! Je ne recomman...,False


## 📚 FASE 3: Manejo de Acentos y Contracciones

In [8]:
!pip install unidecode

   ---------------------------------------- 0.0/235.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/235.8 kB ? eta -:--:--
   ----- --------------------------------- 30.7/235.8 kB 660.6 kB/s eta 0:00:01
   ------------- ------------------------- 81.9/235.8 kB 919.0 kB/s eta 0:00:01
   ---------------------------------------- 235.8/235.8 kB 2.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
### 📌 Fase 3 - Normalización Lingüística

import re
from unidecode import unidecode

# Diccionario de contracciones personalizadas
contracciones = {
    "q'huvo": "que hubo",
    "pq": "porque",
    " x ": " por ",
    "dnd": "donde",
    "k": "que",
    "xq": "porque"
}

# Función para expandir contracciones
def expandir_contracciones(texto):
    for contra, exp in contracciones.items():
        texto = re.sub(r'\b{}\b'.format(re.escape(contra)), exp, texto, flags=re.IGNORECASE)
    return texto

# Función para normalizar hashtags: quitar acento y pasar a minúscula
def normalizar_hashtags(texto):
    def limpiar_hashtag(match):
        hashtag = match.group()
        hashtag = unidecode(hashtag)  # elimina acentos
        return hashtag.lower()
    return re.sub(r'#\w+', limpiar_hashtag, texto)

# Lista de idiomas donde no se elimina acento (por ejemplo: francés, portugués)
idiomas_con_acentos_criticos = ['fr']  # puedes ampliarla si detectas el idioma

# Función de limpieza final
def limpiar_final(texto, idioma='es'):
    if idioma != 'fr':
        texto = unidecode(texto)  # elimina acentos si no es francés
    texto = expandir_contracciones(texto)
    texto = normalizar_hashtags(texto)
    texto = re.sub(r'[^\w\s#😊😞]', '', texto)  # eliminar símbolos no esenciales
    texto = re.sub(r'\s{2,}', ' ', texto)  # eliminar espacios extra
    return texto.strip().lower()

# Aplicación al DataFrame
df['texto_final'] = df['texto_corregido'].apply(lambda x: limpiar_final(x, idioma='es'))  # Cambia idioma si necesario

# Verificación visual
print(df[['texto', 'texto_corregido', 'texto_final']].head())


                                               texto  \
0  Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...   
1  J'adore cet ordinateur! Mais le clavier est tr...   
2  This product is a disaster... don't buy it! #W...   
3  La camisa es bonita, pero se encogiÃÂ³ despuÃ...   
4  Le service clientÃÂ¨le est nul!!! Je ne recom...   

                                     texto_corregido  \
0  Éste celular es increíble!!! Pero tarda 5hs en...   
1  J'adore cet ordinateur! Mais le clavier est tr...   
2  This product is a disaster... don't buy it! #W...   
3  La camisa es bonita, pero se encogió después d...   
4  Le service clientèle est nul!!! Je ne recomman...   

                                         texto_final  
0  este celular es increible pero tarda 5hs en ca...  
1  jadore cet ordinateur mais le clavier est trop...  
2  this product is a disaster dont buy it #wasteo...  
3  la camisa es bonita pero se encogio despues de...  
4  le service clientele est nul je ne recommande pa

## 📈 FASE 4: Evaluación de Impacto

In [23]:
# Asumimos que rating va de 1 (muy malo) a 5 (muy bueno)
# Definimos sentimiento: 1 = negativo (rating 1 o 2), 0 = positivo (rating 3 a 5)
df['sentimiento'] = df['rating'].apply(lambda x: 1 if x <= 2 else 0)

# Verificamos el balance de clases
print(df['sentimiento'].value_counts())


sentimiento
1    19
0    10
Name: count, dtype: int64


In [22]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --------------------------------------------------
# Paso 1: Crear columna 'sentimiento' desde 'rating'
# --------------------------------------------------
# Suponemos: 1 o 2 = negativo (1), 3 a 5 = positivo (0)
df['sentimiento'] = df['rating'].apply(lambda x: 1 if x <= 2 else 0)

# Verificamos cuántos ejemplos hay de cada clase
print("Distribución de clases:")
print(df['sentimiento'].value_counts())

# --------------------------------------------------
# Paso 2: Preparar texto y vectorizadores
# --------------------------------------------------
X_sin_raw = df['texto_corregido']
X_con_raw = df['texto_final']
y = df['sentimiento']

vectorizer_sin = CountVectorizer()
vectorizer_con = CountVectorizer()

vectorizer_sin.fit(X_sin_raw)
vectorizer_con.fit(X_con_raw)

tokens_unicos_sin = len(vectorizer_sin.vocabulary_)
tokens_unicos_con = len(vectorizer_con.vocabulary_)

# --------------------------------------------------
# Paso 3: Función de entrenamiento y evaluación
# --------------------------------------------------
def entrenar_y_evaluar(vectorizer, X_raw, y):
    X = vectorizer.transform(X_raw)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    modelo = LogisticRegression(max_iter=1000)
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    return accuracy_score(y_test, y_pred)

acc_sin = entrenar_y_evaluar(vectorizer_sin, X_sin_raw, y)
acc_con = entrenar_y_evaluar(vectorizer_con, X_con_raw, y)

# --------------------------------------------------
# Paso 4: Tabla de resultados
# --------------------------------------------------
tabla_resultados = pd.DataFrame({
    'Métrica': ['Tokens Únicos', 'Accuracy'],
    'Sin Normalización': [tokens_unicos_sin, f"{acc_sin:.2%}"],
    'Con Normalización': [tokens_unicos_con, f"{acc_con:.2%}"]
})

print("\n📊 Resultados comparativos:")
print(tabla_resultados)


Distribución de clases:
sentimiento
1    19
0    10
Name: count, dtype: int64

📊 Resultados comparativos:
         Métrica Sin Normalización Con Normalización
0  Tokens Únicos               188               191
1       Accuracy            83.33%            83.33%


## 📁  Exportar archivos finales

In [24]:
df.head(3)

,texto,idioma,rating,categoria,texto_corregido,texto_corrupto,texto_final,sentimiento
0,Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...,es,2,electrÃ³nicos,Éste celular es increíble!!! Pero tarda 5hs en...,False,este celular es increible pero tarda 5hs en ca...,1
1,J'adore cet ordinateur! Mais le clavier est tr...,fr,4,electrÃ³nicos,J'adore cet ordinateur! Mais le clavier est tr...,False,jadore cet ordinateur mais le clavier est trop...,0
2,This product is a disaster... don't buy it! #W...,en,1,electrÃ³nicos,This product is a disaster... don't buy it! #W...,False,this product is a disaster dont buy it #wasteo...,1


In [28]:
# Exportar solo la columna 'texto_final' a un archivo CSV
df[['texto_final']].to_csv('outputs/texto_final.csv', index=False)
print("✅ Archivo 'texto_final.csv' exportado correctamente.")


✅ Archivo 'texto_final.csv' exportado correctamente.


In [29]:
# Exportar el DataFrame completo a CSV
df.to_csv('outputs/resenas_normalizadas.csv', index=False)
print("✅ Archivo 'resenas_normalizadas.csv' exportado correctamente con todas las columnas.")


✅ Archivo 'resenas_normalizadas.csv' exportado correctamente con todas las columnas.
